# Clustering Analysis

This notebook implements K-Means clustering with PCA to segment financial data into meaningful groups. We'll identify natural patterns in the data and characterize different performance profiles.

In [ ]:
# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import warnings
import pickle

# Machine learning libraries
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans

# Import custom modules
import sys
import os
sys.path.append(os.path.abspath('../src'))
from ml_utils import plot_elbow_method, plot_pca_components, plot_clusters

# Set plotting style
sns.set(style='whitegrid')
plt.style.use('seaborn-whitegrid')

# Ignore warnings
warnings.filterwarnings('ignore')

# Create directories for saving results if they don't exist
Path('../results/models').mkdir(parents=True, exist_ok=True)
Path('../results/plots/ml').mkdir(parents=True, exist_ok=True)

## 1. Load and Prepare Data

In [ ]:
# Load the cleaned data
df = pd.read_csv('../data/processed/cleaned_data.csv')

# Display basic information
print(f"Dataset shape: {df.shape}")
df.head()

## 2. Feature Selection for Clustering

In [ ]:
# Select features for clustering
# Numeric features for clustering
numeric_features = ['Revenue', 'Cost', 'Profit', 'ROA', 'Profit_Margin']

# Categorical features to encode
categorical_features = ['Segment', 'Country']

# Check if all selected features exist in the dataframe
for feature in numeric_features + categorical_features:
    if feature not in df.columns:
        print(f"Warning: {feature} not found in the dataframe.")

# Create a copy of the data with selected features
clustering_data = df[numeric_features + categorical_features].copy()

# Display the first few rows
clustering_data.head()

## 3. Preprocessing for Clustering

In [ ]:
# Define preprocessing steps
# For numeric features: scaling
# For categorical features: one-hot encoding

# Create preprocessor
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numeric_features),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)
    ])

# Apply preprocessing
X_preprocessed = preprocessor.fit_transform(clustering_data)

# Get feature names after one-hot encoding
ohe = preprocessor.named_transformers_['cat']
cat_feature_names = ohe.get_feature_names_out(categorical_features)
feature_names = numeric_features + list(cat_feature_names)

print(f"Shape after preprocessing: {X_preprocessed.shape}")
print(f"Number of features: {len(feature_names)}")

## 4. Apply PCA for Dimensionality Reduction

In [ ]:
# Apply PCA to reduce dimensionality while retaining 95% of variance
pca = PCA(n_components=0.95)
X_pca = pca.fit_transform(X_preprocessed)

print(f"Shape after PCA: {X_pca.shape}")
print(f"Number of components: {pca.n_components_}")
print(f"Explained variance ratio: {pca.explained_variance_ratio_}")
print(f"Total explained variance: {sum(pca.explained_variance_ratio_):.2f}")

In [ ]:
# Visualize explained variance by components
plot_pca_components(pca, '../results/plots/ml/pca_explained_variance.png')

## 5. Determine Optimal Number of Clusters

In [ ]:
# Use the elbow method to find the optimal number of clusters
k_range = range(2, 15)
optimal_k = plot_elbow_method(X_pca, k_range, '../results/plots/ml/kmeans_elbow_method.png')

print(f"Optimal number of clusters based on elbow method: {optimal_k}")

## 6. Apply K-Means Clustering

In [ ]:
# Apply K-Means clustering with the optimal number of clusters
kmeans = KMeans(n_clusters=optimal_k, random_state=42, n_init=10)
cluster_labels = kmeans.fit_predict(X_pca)

# Add cluster labels to the original dataframe
df['Cluster'] = cluster_labels

# Display the count of data points in each cluster
print("Cluster distribution:")
print(df['Cluster'].value_counts())

In [ ]:
# Visualize clusters in 2D space (using first two PCA components)
plot_clusters(X_pca, cluster_labels, '../results/plots/ml/kmeans_clusters.png')

## 7. Analyze Cluster Characteristics

In [ ]:
# Analyze cluster characteristics
cluster_analysis = df.groupby('Cluster').agg({
    'Revenue': 'mean',
    'Cost': 'mean',
    'Profit': 'mean',
    'ROA': 'mean',
    'Profit_Margin': 'mean',
    'Cluster': 'count'
}).rename(columns={'Cluster': 'Count'}).sort_values('Profit', ascending=False)

print("Cluster characteristics:")
cluster_analysis

In [ ]:
# Visualize cluster characteristics
plt.figure(figsize=(14, 10))

# Plot metrics by cluster
metrics = ['Revenue', 'Cost', 'Profit', 'ROA', 'Profit_Margin']
for i, metric in enumerate(metrics):
    plt.subplot(2, 3, i+1)
    sns.barplot(x='Cluster', y=metric, data=df)
    plt.title(f'Average {metric} by Cluster')
    plt.grid(axis='y')

plt.tight_layout()
plt.savefig('../results/plots/ml/cluster_characteristics.png')
plt.show()

In [ ]:
# Analyze segment distribution across clusters
segment_cluster = pd.crosstab(df['Segment'], df['Cluster'], normalize='index') * 100

plt.figure(figsize=(14, 8))
segment_cluster.plot(kind='bar', stacked=True)
plt.title('Segment Distribution Across Clusters')
plt.xlabel('Segment')
plt.ylabel('Percentage (%)')
plt.legend(title='Cluster')
plt.grid(axis='y')
plt.tight_layout()
plt.savefig('../results/plots/ml/segment_cluster_distribution.png')
plt.show()

## 8. Assign Meaningful Cluster Labels

In [ ]:
# Assign meaningful labels to clusters based on their characteristics
def assign_cluster_labels(cluster_analysis):
    # Sort clusters by profit (high to low)
    profit_order = cluster_analysis.sort_values('Profit', ascending=False).index.tolist()
    
    # Sort clusters by ROA (high to low)
    roa_order = cluster_analysis.sort_values('ROA', ascending=False).index.tolist()
    
    # Sort clusters by profit margin (high to low)
    margin_order = cluster_analysis.sort_values('Profit_Margin', ascending=False).index.tolist()
    
    # Create labels dictionary
    cluster_labels = {}
    
    for cluster in cluster_analysis.index:
        # Determine profit level
        profit_rank = profit_order.index(cluster)
        profit_level = "High Profit" if profit_rank < len(profit_order) / 3 else \
                      "Medium Profit" if profit_rank < 2 * len(profit_order) / 3 else "Low Profit"
        
        # Determine ROA level
        roa_rank = roa_order.index(cluster)
        roa_level = "High ROA" if roa_rank < len(roa_order) / 3 else \
                   "Medium ROA" if roa_rank < 2 * len(roa_order) / 3 else "Low ROA"
        
        # Combine labels
        cluster_labels[cluster] = f"{profit_level}/{roa_level}"
    
    return cluster_labels

# Get cluster labels
cluster_labels_dict = assign_cluster_labels(cluster_analysis)
print("Cluster labels:")
for cluster, label in cluster_labels_dict.items():
    print(f"Cluster {cluster}: {label}")

# Add descriptive labels to the dataframe
df['Cluster_Label'] = df['Cluster'].map(cluster_labels_dict)

# Display the count of data points in each labeled cluster
print("\nLabeled cluster distribution:")
print(df['Cluster_Label'].value_counts())

In [ ]:
# Save the clustering model and preprocessor
clustering_model = {
    'preprocessor': preprocessor,
    'pca': pca,
    'kmeans': kmeans,
    'cluster_labels': cluster_labels_dict
}

with open('../results/models/clustering_model.pkl', 'wb') as f:
    pickle.dump(clustering_model, f)

print("Clustering model saved.")

## 9. Cluster Insights and Recommendations

In [ ]:
# Summarize cluster characteristics with labels
labeled_cluster_analysis = df.groupby('Cluster_Label').agg({
    'Revenue': 'mean',
    'Cost': 'mean',
    'Profit': 'mean',
    'ROA': 'mean',
    'Profit_Margin': 'mean',
    'Cluster_Label': 'count'
}).rename(columns={'Cluster_Label': 'Count'}).sort_values('Profit', ascending=False)

print("Labeled cluster characteristics:")
labeled_cluster_analysis

### Clustering Insights:

1. We identified {optimal_k} distinct financial performance clusters, each with unique characteristics.
2. The "High Profit/High ROA" cluster represents our top-performing segment with exceptional profitability and return on assets.
3. The "Low Profit/Low ROA" cluster indicates areas that need immediate attention and strategic intervention.
4. Certain segments show a strong presence in high-performing clusters, suggesting industry-specific advantages.
5. The clustering analysis reveals natural groupings that go beyond simple profitability metrics, providing a multi-dimensional view of financial performance.

### Recommendations based on Clustering:

1. **Strategic Focus on High-Potential Segments**: Allocate more resources to segments that appear frequently in the "High Profit/High ROA" cluster.
2. **Targeted Interventions for Underperforming Units**: Develop specific strategies for entities in the "Low Profit/Low ROA" cluster, focusing on improving the key metrics identified in our analysis.
3. **Performance Benchmarking**: Establish cluster-based benchmarks for different business units, recognizing that different segments may have different performance profiles.
4. **Segment-Specific Strategies**: Develop tailored approaches for each segment based on their cluster distribution patterns.
5. **Cross-Segment Learning**: Identify best practices from segments that consistently appear in high-performing clusters and apply them to underperforming segments where applicable.

## 10. Save Results for Further Analysis

In [ ]:
# Save the dataframe with cluster labels for use in other notebooks
df.to_csv('../data/processed/clustered_data.csv', index=False)

# Save cluster analysis summary
labeled_cluster_analysis.to_csv('../results/reports/cluster_analysis_summary.csv')

print("Results saved for further analysis.")